# Models visualised

In [ ]:
import datetime as dt
import logging
import pathlib

import numpy as np
import xarray as xr
import pandas as pd
# import matplotlib as mpl
# import matplotlib.pyplot as plt
# from scipy.interpolate import interp1d
# import cartopy.crs as ccrs
# import cartopy.feature as cfeature
import holoviews as hv
import hvplot.xarray
import panel as pn

import eoxmagmod
from chaosmagpy.plot_utils import nio_colormap

logger = logging.getLogger("model-explorer")

In [ ]:
hv.extension("bokeh")
pn.extension("mathjax")

## Clear cache if necessary

Should be in `./cache`. The `@pn.cache(to_disk=True)` decorators further down populate this cache.

In [ ]:
# pn.state.clear_caches()

## Magnetic model evaluation tools

https://esa-vires.github.io/MagneticModel/index.html

In [ ]:
try:
    current_folder = pathlib.Path(__file__).parent
except NameError:
    current_folder = globals()['_dh'][0]

In [ ]:
def generate_latlon_grid(resolution=2, min_lat=-90, max_lat=90, height=0):
    "Generate a grid of positions over the Earth at a given degree resolution at a given height (km) relative to reference radius"
    lat, lon = np.meshgrid(
        np.arange(min_lat, max_lat, resolution),
        np.arange(-180, 180, resolution)
    )
    REFRAD_KM = 6371.200
    radius = np.ones_like(lat)*(height + REFRAD_KM)
    return lat, lon, radius


def datetime_to_mjd2000(t: dt.datetime) -> float:
    "Convert a datetime object to MJD2000."
    # Convert to datetime64 ns
    t = np.datetime64(t).astype("M8[ns]")
    # Get offset to year 2000
    t = (t - np.datetime64('2000')).astype('int64')
    # Convert from ns to days
    NS2DAYS = 1.0/(24*60*60*1e9)
    return t * NS2DAYS


def eval_model(
        lat: np.ndarray, lon: np.ndarray, radius: np.ndarray,
        times=np.array([dt.datetime(2018, 1, 1)]),
        shc_model=eoxmagmod.data.IGRF13,
        generic_model=None, f107=1.0
    ):
    """Evaluate a model on a grid across different times

    Returns:
        ndarray: B_NEC values at each point
    """
    # Convert Python datetime to MJD2000
    mjd2000 = [datetime_to_mjd2000(t) for t in times]
    # Load model
    model = generic_model if generic_model else eoxmagmod.load_model_shc(shc_model)
    # Reshape the input coordinates to use eoxmagmod
    orig_shape = lat.shape
    _lat, _lon, _radius = map(lambda x: x.flatten(), (lat, lon, radius))
    coords = np.stack((_lat, _lon, _radius), axis=1)
    # expand coords according to the time dimension
    coords = np.stack([coords for i in range(len(mjd2000))])
    timestack = np.stack([np.ones_like(_lat)*t for t in mjd2000])
    # Evaluate in North, East, Up coordinates
    # Specify coordinate systems according to:
    #   CT_GEODETIC_ABOVE_WGS84 = 0,
    #   CT_GEOCENTRIC_SPHERICAL = 1,
    #   CT_GEOCENTRIC_CARTESIAN = 2
    # https://github.com/ESA-VirES/MagneticModel/blob/staging/eoxmagmod/eoxmagmod/pymm_coord.h
    input_coordinate_system = 1
    output_coordinate_system = 1
    # Use scale to convert NEU to NEC
    b_nec = model.eval(timestack, coords, input_coordinate_system, output_coordinate_system, scale=[1, 1, -1], f107=f107)
    # Inclination (I), declination (D), intensity (F)
    # inc, dec, F = eoxmagmod.vincdecnorm(b_neu)
    # Reshape output back to original grid
    b_nec = b_nec.reshape(times.shape + orig_shape + (3, ))
    return b_nec


def eval_model_on_grid(model=None, heights=np.arange(0, 1000, 100), resolution=1, times=np.array([dt.datetime(2018, 1, 1)])):
    ds_set = []
    for height in heights:
        latG, lonG, radiusG = generate_latlon_grid(resolution=resolution, height=height)
        b_nec = eval_model(latG, lonG, radiusG, times=times, generic_model=model)
        _ds = xr.Dataset({'B_NEC': (['time', 'x', 'y', 'NEC'],  b_nec)},
                 coords={'lon': (['x', 'y'], lonG),
                         'lat': (['x', 'y'], latG),
                         'time': times})
        _ds = _ds.assign_coords({"height": height})
        ds_set += [_ds]
    ds = xr.concat(ds_set, dim="height")
    ds["height"].attrs = {"units": "km"}
    ds = ds.assign_coords({"NEC": np.array(["N", "E", "C"])})
    return ds

## Core field

In [ ]:
@pn.cache(to_disk=True)
def get_core_field():
    logger.warning("Evaluating core field...")
    mco_sha_2c_file = current_folder / pathlib.Path("./data/SW_OPER_MCO_SHA_2C_20131125T000000_20220601T000000_0801/SW_OPER_MCO_SHA_2C_20131125T000000_20220601T000000_0801.shc")
    mco = eoxmagmod.load_model_shc(str(mco_sha_2c_file))
    return eval_model_on_grid(
        model=mco,
        heights=np.arange(-3000, 3000, 500),
        resolution=0.5,
    )

ds_core = get_core_field()
# ds_core

In [ ]:
def plot_slice_core(height=0, itime=0, NEC="C"):
    return ds_core.isel(time=0).sel(height=height, NEC=NEC).hvplot.scatter(
        x="lon", y="lat", c="B_NEC", by="NEC", subplots=True,
        rasterize=True, colorbar=True, hover=False, width=500, height=250, cmap=nio_colormap(), dynamic=False,
        # clabel="nT"
    ).opts(shared_axes=False)

dmap_core = (
    hv.DynamicMap(plot_slice_core, kdims=["height"])#, "NEC"])
    .redim.values(height=tuple(ds_core["height"].values),)# NEC=("N", "E", "C"))
).options(title="Core (MCO)", show_title=False)
# dmap_core

In [ ]:
# height_slider = pn.widgets.DiscreteSlider(options=list(ds_igrf["height"].values), name="height", value=0)
# dmap = hv.DynamicMap(plot_slice_core, kdims=["height"], streams=[hv.streams.Params(height_slider, ['height'])])
# pn.Column(height_slider, dmap)

## Lithospheric field

In [ ]:
@pn.cache(to_disk=True)
def get_crust_field():
    logger.warning("Evaluating crustal field...")
    mli_sha_2c_file = current_folder / pathlib.Path("./data/SW_OPER_MLI_SHA_2C_00000000T000000_99999999T999999_0801/SW_OPER_MLI_SHA_2C_00000000T000000_99999999T999999_0801.shc")
    mli = eoxmagmod.load_model_shc(str(mli_sha_2c_file))
    return eval_model_on_grid(
        model=mli,
        heights=range(0, 600, 100),
        resolution=0.5
    )

ds_crust = get_crust_field()
# ds_crust

In [ ]:
def plot_slice_crust(height=0, itime=0, NEC="C"):
    _ds = ds_crust.sel(height=height, NEC=NEC).isel(time=itime)
    minmax = float(min(abs(_ds["B_NEC"].quantile(0.01)), abs(_ds["B_NEC"].quantile(0.99))))
    return _ds.hvplot.scatter(
        x="lon", y="lat", c="B_NEC", by="NEC", subplots=True,
        rasterize=True, colorbar=True, dynamic=False, hover=False,
        width=500, height=250, cmap=nio_colormap(), clim=(-minmax, minmax),
    ).opts(shared_axes=False)

In [ ]:
dmap_crust = (
    hv.DynamicMap(plot_slice_crust, kdims=["height",])
    .redim.values(height=tuple(ds_crust["height"].values),)
).options(title="Crust (MLI)", show_title=False)
# dmap_crust

## Ionospheric field

In [ ]:
@pn.cache(to_disk=True)
def get_mio_field():
    logger.warning("Evaluating ionospheric field...")
    mio_sha_2c_file = current_folder / pathlib.Path("./data/SW_OPER_MIO_SHA_2C_00000000T000000_99999999T999999_0801/SW_OPER_MIO_SHA_2C_00000000T000000_99999999T999999_0801.txt")
    mio_e = eoxmagmod.load_model_swarm_mio_external(str(mio_sha_2c_file))
    mio_i = eoxmagmod.load_model_swarm_mio_internal(str(mio_sha_2c_file))
    heights = np.array([0])
    times = np.array([dt.datetime(2018, 1, 1, x) for x in range(0, 24, 2)])
    resolution = 0.5
    ds_mio_e = eval_model_on_grid(model=mio_e, heights=heights, times=times, resolution=resolution)
    ds_mio_i = eval_model_on_grid(model=mio_i, heights=heights, times=times, resolution=resolution)
    return xr.Dataset(
        {"B_NEC":
             xr.concat(
                (ds_mio_e["B_NEC"].expand_dims(dim="component"), ds_mio_i["B_NEC"].expand_dims(dim="component")),
                dim="component"
            ).assign_coords({"component": np.array(["external", "internal"])})
        }
    )

ds_mio = get_mio_field()
# ds_mio_e

In [ ]:
def plot_slice_mio(height=0, time=0, component="external", NEC="C"):
    _ds = ds_mio.sel(height=height, time=time, component=component, NEC=NEC)
    minmax = float(min(abs(_ds["B_NEC"].quantile(0.01)), abs(_ds["B_NEC"].quantile(0.99))))
    return _ds.hvplot.scatter(
        x="lon", y="lat", c="B_NEC", by="NEC", subplots=True,
        rasterize=True, colorbar=True, dynamic=False, hover=False,
        width=500, height=250, cmap=nio_colormap(), clim=(-minmax, minmax),
    ).opts(shared_axes=False)

dmap_mio = (
    hv.DynamicMap(plot_slice_mio, kdims=["time", "component"])
    .redim.values(time=tuple(ds_mio["time"].values), component=("external", "internal"))
).options(title="Ionosphere (MIO)", show_title=False)
# dmap_mio

## Magnetospheric field

In [ ]:
@pn.cache(to_disk=True)
def get_mma_field():
    logger.warning("Evaluating magnetospheric field...")
    mma_sha_2c_file = current_folder / pathlib.Path("./data/SW_OPER_MMA_SHA_2C_20131125T000000_20181231T235959_0501/SW_OPER_MMA_SHA_2C_20131125T000000_20181231T235959_0501.cdf")
    mma_e = eoxmagmod.load_model_swarm_mma_2c_external(str(mma_sha_2c_file))
    mma_i = eoxmagmod.load_model_swarm_mma_2c_internal(str(mma_sha_2c_file))
    heights = np.array([0])
    times = np.array([dt.datetime(2018, 1, 1, x) for x in range(0, 24, 2)])
    resolution = 0.5
    ds_mma_e = eval_model_on_grid(model=mma_e, heights=heights, times=times, resolution=resolution)
    ds_mma_i = eval_model_on_grid(model=mma_i, heights=heights, times=times, resolution=resolution)
    return xr.Dataset(
        {"B_NEC":
             xr.concat(
                (ds_mma_e["B_NEC"].expand_dims(dim="component"), ds_mma_i["B_NEC"].expand_dims(dim="component")),
                dim="component"
            ).assign_coords({"component": np.array(["external", "internal"])})
        }
    )

ds_mma = get_mma_field()
# ds_mma

In [ ]:
def plot_slice_mma(height=0, time=0, component="external", NEC="C"):
    _ds = ds_mma.sel(height=height, time=time, component=component, NEC=NEC)
    minmax = float(min(abs(_ds["B_NEC"].quantile(0.01)), abs(_ds["B_NEC"].quantile(0.99))))
    return _ds.hvplot.scatter(
        x="lon", y="lat", c="B_NEC", by="NEC", subplots=True,
        rasterize=True, colorbar=True, dynamic=False, hover=False,
        width=500, height=250, cmap=nio_colormap(), clim=(-minmax, minmax),
    ).opts(shared_axes=False)

dmap_mma = (
    hv.DynamicMap(plot_slice_mma, kdims=["time", "component"])
    .redim.values(time=tuple(ds_mma["time"].values), component=("external", "internal"))
).options(title="Magnetosphere (MMA)", show_title=False)
# dmap_mma

## Compile dashboard

In [ ]:
text = r"""
## Models of the geomagnetic field

These figures show the **vertical component (in nT)** of the four largest aspects of the ***quiet-time* geomagnetic field**. They demonstrate the main parts of the "Comprehensive Model", which co-estimates the fields from different sources, parameterised using spherical harmonics. See for more info:

Sabaka, T.J., Tøffner-Clausen, L., Olsen, N. et al. CM6: a comprehensive geomagnetic field model derived from both CHAMP and Swarm satellite observations. Earth Planets Space 72, 80 (2020). https://doi.org/10.1186/s40623-020-01210-5

The models are delivered through the Swarm framework as the `SwarmCI` models: [`MCO_SHA_2C`](https://swarmhandbook.earth.esa.int/catalogue/SW_MCO_SHA_2C) (core), [`MLI_SHA_2C`](https://swarmhandbook.earth.esa.int/catalogue/SW_MLI_SHA_2C) (lithosphere), [`MIO_SHA_2C`](https://swarmhandbook.earth.esa.int/catalogue/SW_MIO_SHA_2C) (ionosphere), [`MMA_SHA_2C`](https://swarmhandbook.earth.esa.int/catalogue/SW_MMA_SHA_2C) (magnetosphere), excluding the oceanic tidal components.

### Core and lithosphere:

From Sabaka et al. 2020:
> The core and lithospheric magnetic fields are expressed as gradients of solid harmonic (SH) expansions of the internal branch solution to Laplace’s equation in spherical coordinates corresponding to an SH degree truncation level of $$N_{max}=120$$, where the first $$N_{SV}=18$$ degrees allow for secular variation (SV) in the form of order-4 B-splines spanning 1999.0 to 2019.5 with knots every 6 months giving a total of 45 parameters per SH coefficient, and for degrees above $$N_{SV}$$ the coefficients are constant. The expression for the corresponding core/lithospheric potential at time $$t$$ and position $$\vec{r}$$, corresponding to Earth-Centered Earth-Fixed (ECEF) spherical coordinates of radius, colatitude, and longitude $$(r,\theta,\phi)$$, is given by
>
> $$V_{cl}(t,\vec{r}) = \Re \left( a \sum_{n=1}^{120} \sum_{m=0}^{n} \left(\frac{a}{r}\right)^{n+1} (\gamma^m_n (t))^* Y^m_n (\theta, \phi) \right)$$

See the paper for more details. The SH coefficients are expressed compactly as $$\gamma^m_n (t) = g^m_n(t) + ih^m_n(t)$$ and are time-dependent for $$n \le 18$$ (core), and time-independent for $$n \ge 19$$ (lithosphere).

**The visualisations display how the field varies with distance to their respective sources. Much more structure is visible in the core field when displayed closer to the CMB, but becomes dipolar when viewed from further away. The lithospheric field is much weaker, especially when viewed from satellite height (~500km).**

### Ionosphere:

This part parameterises the mid-latitude ionospheric field originating mainly from the *Sq* system, up to $$n=60, m=12$$, in *Quasi-Dipole coordinates*, modulated by seasonal and daily wavenumbers and the F10.7 index. This results in a similar cyclical pattern repeating each day, and each year, with its intensity varying with solar output. The primary (external) field induces a secondary (internal) counterpart in the mantle, the model for which is further dependent on a mantle conductivity model.

**The visualisation shows the variation of the field over one day, peaking in intensity around local noon.**

### Magnetosphere:

This part of the model estimates the effect, near Earth, of more remote magnetospheric sources, mainly from the ring current and magnetopause currents. Similar to the ionospheric part, it also parameterises the induced counterpart. The field is discretised into 1-hour time bins (the field is treated as static over this time period), expressed in SH up to $$n=1, m=1$$ in *dipole coordinates*.
"""

In [ ]:
gspec = pn.GridSpec(width=1600, height=1100)
gspec[0, 0] = dmap_core
gspec[1, 0] = dmap_crust
gspec[2, 0] = dmap_mio
gspec[3, 0] = dmap_mma
gspec[:, 1] = pn.pane.Markdown(text)
gspec.servable(title="Geomagnetic Model Explorer")